In [56]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [57]:
pricing_data = web.DataReader("NVDA", "yahoo")
closing_price = pd.DataFrame(pricing_data["Close"])

# training_set, testing_set = pd.Series(train_test_split(closing_price.dropna(), test_size = 0.20, random_state = None, shuffle = False))

training_set, testing_set = closing_price[0:-20], closing_price[-20:]

In [58]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [59]:
X_train = []
y_train = []
for i in range(10, training_set.shape[0]):
    X_train.append(training_set_scaled[i-10:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [60]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
39/39 [==============================] - 4s 9ms/step - loss: 0.0213
Epoch 2/100
39/39 [==============================] - 0s 9ms/step - loss: 0.0030
Epoch 3/100
39/39 [==============================] - 0s 8ms/step - loss: 0.0027
Epoch 4/100
39/39 [==============================] - 0s 8ms/step - loss: 0.0026
Epoch 5/100
39/39 [==============================] - 0s 9ms/step - loss: 0.0024
Epoch 6/100
39/39 [==============================] - 0s 9ms/step - loss: 0.0023
Epoch 7/100
39/39 [==============================] - 0s 9ms/step - loss: 0.0023
Epoch 8/100
39/39 [==============================] - 0s 10ms/step - loss: 0.0022
Epoch 9/100
39/39 [==============================] - 0s 9ms/step - loss: 0.0024
Epoch 10/100
39/39 [==============================] - 0s 9ms/step - loss: 0.0021
Epoch 11/100
39/39 [==============================] - 0s 9ms/step - loss: 0.0023
Epoch 12/100
39/39 [==============================] - 0s 8ms/step - loss: 0.0019
Epoch 13/100
39/39 [================

In [61]:
dataset_total = pd.concat((training_set, testing_set), axis = 0)
inputs = dataset_total[len(dataset_total) - len(testing_set) - 10:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(10, testing_set.shape[0]):
    X_test.append(inputs[i-10:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 1s 822ms/step


In [62]:
predicted_stock_price

array([[130.27283 ],
       [127.8049  ],
       [126.119316],
       [126.04184 ],
       [125.66931 ],
       [124.83469 ],
       [124.75995 ],
       [126.659225],
       [129.3018  ],
       [131.35002 ]], dtype=float32)

In [63]:
# # Testing set plot
# plt.plot(testing_set.reset_index(drop = True), color = 'black', label = 'TATA Stock Price')
# plt.plot(predicted_stock_price, color = 'green', label = 'Predicted TATA Stock Price')
# plt.title('TATA Stock Price Prediction')
# plt.xlabel('Time')
# plt.ylabel('TATA Stock Price')
# plt.legend()
# plt.show()